<a href="https://colab.research.google.com/github/Sowmithaa22/NLP-Project/blob/main/nlp_cia_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Raw OG Code


In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from heapq import nlargest
import nltk

nltk.download('punkt')  # Ensure the punkt tokenizer is downloaded

class BeamSearchSummarizer:
    def __init__(self, text, beam_width=3, summary_length=5):
        self.sentences = nltk.sent_tokenize(text)  # Tokenize the paragraph into sentences
        self.beam_width = beam_width
        self.summary_length = summary_length

    def cluster_sentences(self, num_clusters):
        """Cluster sentences and return the most important sentence from each cluster"""
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(self.sentences)
        kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10).fit(X)
        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_
        key_sentences = []

        for cluster in range(num_clusters):
            indices = np.where(labels == cluster)[0]
            cluster_sentences = [self.sentences[i] for i in indices]
            cluster_vectors = X[indices].toarray()
            centroid_vector = centroids[cluster].reshape(1, -1)
            similarities = cosine_similarity(cluster_vectors, centroid_vector).flatten()
            top_sentence_index = indices[np.argmax(similarities)]
            key_sentences.append(self.sentences[top_sentence_index])

        return key_sentences

    def beam_search_summary(self):
        """Generate a summary using Beam Search"""
        num_clusters = min(len(self.sentences), self.summary_length)
        key_sentences = self.cluster_sentences(num_clusters)
        beams = [([], 0)]  # Initialize with empty list and score

        for sentence in key_sentences:
            new_beams = []
            for beam_list, beam_score in beams:
                new_list = beam_list + [sentence]
                score = self.evaluate_summary(new_list)
                new_beams.append((new_list, score))

            # Keep only the top 'beam_width' candidates
            beams = nlargest(self.beam_width, new_beams, key=lambda x: x[1])

        # Select the best-scoring sequence as final summary
        best_summary = beams[0][0] if beams else []  # Extract best sequence of sentences
        return " ".join(best_summary)

    def evaluate_summary(self, summary):
        """Evaluation function to rank summaries"""
        if not summary:  # Handle empty summary case
            return 0

        length_penalty = abs(len(summary) - self.summary_length)
        # Count repeated sentences in the list
        redundancy_penalty = sum(summary.count(s) - 1 for s in set(summary))
        unique_sentences = len(set(summary))
        return unique_sentences - length_penalty - redundancy_penalty

# Example Usage:
if __name__ == "__main__":
    text = """
    Artificial Intelligence (AI) has revolutionized various aspects of human life, from automation to decision-making. It is a rapidly evolving field that integrates machine learning, deep learning, and natural language processing to mimic human intelligence. AI has had a significant impact on four key areas: healthcare, business, education, and ethics. In healthcare, AI-powered diagnostic tools have improved the accuracy and speed of disease detection. Machine learning models analyze medical images, detect patterns, and assist doctors in making precise diagnoses. Additionally, AI-driven robotic surgeries and personalized treatment plans are enhancing patient care and medical outcomes. In the business sector, AI has streamlined operations through automation and predictive analytics. Companies use AI-powered chatbots for customer service, fraud detection in banking, and demand forecasting for supply chain management. AI-driven insights help businesses make data-driven decisions, increasing efficiency and profitability. Education has also been transformed by AI, making learning more accessible and personalized. AI-based tutoring systems adapt to individual learning paces, while automated grading reduces the burden on teachers. Virtual reality (VR) and AI-powered simulations provide immersive learning experiences, bridging the gap between theory and practice. Despite its benefits, AI raises ethical concerns regarding privacy, bias, and job displacement. Algorithms can unintentionally reinforce biases present in data, leading to unfair decisions. The automation of jobs also threatens employment in certain industries, necessitating policies for workforce reskilling. Ethical AI development requires transparency, accountability, and human oversight to ensure fairness and safety. AI continues to shape the future, influencing multiple domains. While it brings immense benefits, responsible development and regulation are necessary to harness its potential while mitigating risks.
    """

    summarizer = BeamSearchSummarizer(text, beam_width=3, summary_length=5)
    summary = summarizer.beam_search_summary()
    print("Generated Summary:\n", summary)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Generated Summary:
 In healthcare, AI-powered diagnostic tools have improved the accuracy and speed of disease detection. Education has also been transformed by AI, making learning more accessible and personalized. AI-driven insights help businesses make data-driven decisions, increasing efficiency and profitability. It is a rapidly evolving field that integrates machine learning, deep learning, and natural language processing to mimic human intelligence. Despite its benefits, AI raises ethical concerns regarding privacy, bias, and job displacement.


Step wise Output Code

1. TF-IDF

In [9]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from heapq import nlargest
import nltk

nltk.download('punkt')  # Ensure the punkt tokenizer is downloaded
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [10]:

class BeamSearchSummarizer:
    def __init__(self, text, beam_width=3, summary_length=5):
        self.sentences = nltk.sent_tokenize(text)  # Tokenize the paragraph into sentences
        self.beam_width = beam_width
        self.summary_length = summary_length
        print("\n=== Step 1: Sentence Tokenization ===")
        for i, sentence in enumerate(self.sentences):
            print(f"Sentence {i+1}: {sentence}")

    def cluster_sentences(self, num_clusters):
        """Cluster sentences and return the most important sentence from each cluster"""
        print("\n=== Step 2: TF-IDF Vectorization ===")
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(self.sentences)
        print(f"TF-IDF Matrix Shape: {X.shape}")

        print("\n=== Step 3: K-Means Clustering ===")
        kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10).fit(X)
        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_
        print(f"Cluster Assignments: {labels}")

        key_sentences = []
        print("\n=== Step 4: Identifying Key Sentences from Clusters ===")
        for cluster in range(num_clusters):
            indices = np.where(labels == cluster)[0]
            cluster_sentences = [self.sentences[i] for i in indices]
            cluster_vectors = X[indices].toarray()
            centroid_vector = centroids[cluster].reshape(1, -1)
            similarities = cosine_similarity(cluster_vectors, centroid_vector).flatten()
            top_sentence_index = indices[np.argmax(similarities)]
            key_sentence = self.sentences[top_sentence_index]
            key_sentences.append(key_sentence)
            print(f"Cluster {cluster+1}: Selected Sentence -> {key_sentence}")

        return key_sentences

    def beam_search_summary(self):
        """Generate a summary using Beam Search"""
        num_clusters = min(len(self.sentences), self.summary_length)
        key_sentences = self.cluster_sentences(num_clusters)

        print("\n=== Step 5: Beam Search Process ===")
        beams = [([], 0)]  # Initialize with empty list and score

        for sentence in key_sentences:
            print(f"\nAdding sentence: {sentence}")
            new_beams = []
            for beam_list, beam_score in beams:
                new_list = beam_list + [sentence]
                score = self.evaluate_summary(new_list)
                new_beams.append((new_list, score))
                print(f"Candidate Summary: {' '.join(new_list)} | Score: {score}")

            # Keep only the top 'beam_width' candidates
            beams = nlargest(self.beam_width, new_beams, key=lambda x: x[1])
            print(f"Top {self.beam_width} Candidates After This Step:")
            for b in beams:
                print(f"  {' '.join(b[0])} | Score: {b[1]}")

        # Select the best-scoring sequence as final summary
        best_summary = beams[0][0] if beams else []  # Extract best sequence of sentences
        print("\n=== Step 6: Final Selected Summary ===")
        print(" ".join(best_summary))
        return " ".join(best_summary)

    def evaluate_summary(self, summary):
        """Evaluation function to rank summaries"""
        if not summary:  # Handle empty summary case
            return 0

        length_penalty = abs(len(summary) - self.summary_length)
        redundancy_penalty = sum(summary.count(s) - 1 for s in set(summary))
        unique_sentences = len(set(summary))
        return unique_sentences - length_penalty - redundancy_penalty



In [11]:
# Example Usage:
if __name__ == "__main__":
    text = """
    Artificial Intelligence (AI) has revolutionized various aspects of human life, from automation to decision-making. It is a rapidly evolving field that integrates machine learning, deep learning, and natural language processing to mimic human intelligence. AI has had a significant impact on four key areas: healthcare, business, education, and ethics. In healthcare, AI-powered diagnostic tools have improved the accuracy and speed of disease detection. Machine learning models analyze medical images, detect patterns, and assist doctors in making precise diagnoses. Additionally, AI-driven robotic surgeries and personalized treatment plans are enhancing patient care and medical outcomes. In the business sector, AI has streamlined operations through automation and predictive analytics. Companies use AI-powered chatbots for customer service, fraud detection in banking, and demand forecasting for supply chain management. AI-driven insights help businesses make data-driven decisions, increasing efficiency and profitability. Education has also been transformed by AI, making learning more accessible and personalized. AI-based tutoring systems adapt to individual learning paces, while automated grading reduces the burden on teachers. Virtual reality (VR) and AI-powered simulations provide immersive learning experiences, bridging the gap between theory and practice. Despite its benefits, AI raises ethical concerns regarding privacy, bias, and job displacement. Algorithms can unintentionally reinforce biases present in data, leading to unfair decisions. The automation of jobs also threatens employment in certain industries, necessitating policies for workforce reskilling. Ethical AI development requires transparency, accountability, and human oversight to ensure fairness and safety. AI continues to shape the future, influencing multiple domains. While it brings immense benefits, responsible development and regulation are necessary to harness its potential while mitigating risks.
    """

    summarizer = BeamSearchSummarizer(text, beam_width=3, summary_length=5)
    summary = summarizer.beam_search_summary()
    print("\n=== Final Generated Summary ===\n", summary)



=== Step 1: Sentence Tokenization ===
Sentence 1: 
    Artificial Intelligence (AI) has revolutionized various aspects of human life, from automation to decision-making.
Sentence 2: It is a rapidly evolving field that integrates machine learning, deep learning, and natural language processing to mimic human intelligence.
Sentence 3: AI has had a significant impact on four key areas: healthcare, business, education, and ethics.
Sentence 4: In healthcare, AI-powered diagnostic tools have improved the accuracy and speed of disease detection.
Sentence 5: Machine learning models analyze medical images, detect patterns, and assist doctors in making precise diagnoses.
Sentence 6: Additionally, AI-driven robotic surgeries and personalized treatment plans are enhancing patient care and medical outcomes.
Sentence 7: In the business sector, AI has streamlined operations through automation and predictive analytics.
Sentence 8: Companies use AI-powered chatbots for customer service, fraud detectio

2. WORD2VEC

In [12]:
import nltk
import numpy as np
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from heapq import nlargest

class BeamSearchSummarizer:
    def __init__(self, text, word2vec_model, beam_width=3, summary_length=5):
        self.sentences = nltk.sent_tokenize(text)  # Tokenize paragraph into sentences
        self.word2vec_model = word2vec_model
        self.beam_width = beam_width
        self.summary_length = summary_length

    def get_sentence_vector(self, sentence):
        words = [word for word in nltk.word_tokenize(sentence.lower()) if word.isalpha()]
        vectors = [self.word2vec_model[word] for word in words if word in self.word2vec_model]
        return np.mean(vectors, axis=0) if vectors else np.zeros(self.word2vec_model.vector_size)

    def cluster_sentences(self, num_clusters):
        sentence_vectors = np.array([self.get_sentence_vector(sentence) for sentence in self.sentences])

        kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10).fit(sentence_vectors)
        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_

        key_sentences = []
        for cluster in range(num_clusters):
            indices = np.where(labels == cluster)[0]
            cluster_vectors = sentence_vectors[indices]
            centroid_vector = centroids[cluster].reshape(1, -1)
            similarities = cosine_similarity(cluster_vectors, centroid_vector).flatten()
            top_sentence_index = indices[np.argmax(similarities)]
            key_sentences.append(self.sentences[top_sentence_index])

        return key_sentences

    def beam_search_summary(self):
        num_clusters = min(len(self.sentences), self.summary_length)
        key_sentences = self.cluster_sentences(num_clusters)

        beams = [([], 0)]  # Initialize with empty list and score
        for sentence in key_sentences:
            new_beams = []
            for beam_list, beam_score in beams:
                new_list = beam_list + [sentence]
                score = self.evaluate_summary(new_list)
                new_beams.append((new_list, score))

            beams = nlargest(self.beam_width, new_beams, key=lambda x: x[1])

        best_summary = beams[0][0] if beams else []
        return " ".join(best_summary)

    def evaluate_summary(self, summary):
        if not summary:
            return 0
        length_penalty = abs(len(summary) - self.summary_length)
        redundancy_penalty = sum(summary.count(s) - 1 for s in set(summary))
        unique_sentences = len(set(summary))
        return unique_sentences - length_penalty - redundancy_penalty

#Example Usage:
text = """
    Artificial Intelligence (AI) has revolutionized various aspects of human life, from automation to decision-making. It is a rapidly evolving field that integrates machine learning, deep learning, and natural language processing to mimic human intelligence. AI has had a significant impact on four key areas: healthcare, business, education, and ethics. In healthcare, AI-powered diagnostic tools have improved the accuracy and speed of disease detection. Machine learning models analyze medical images, detect patterns, and assist doctors in making precise diagnoses. Additionally, AI-driven robotic surgeries and personalized treatment plans are enhancing patient care and medical outcomes. In the business sector, AI has streamlined operations through automation and predictive analytics. Companies use AI-powered chatbots for customer service, fraud detection in banking, and demand forecasting for supply chain management. AI-driven insights help businesses make data-driven decisions, increasing efficiency and profitability. Education has also been transformed by AI, making learning more accessible and personalized. AI-based tutoring systems adapt to individual learning paces, while automated grading reduces the burden on teachers. Virtual reality (VR) and AI-powered simulations provide immersive learning experiences, bridging the gap between theory and practice. Despite its benefits, AI raises ethical concerns regarding privacy, bias, and job displacement. Algorithms can unintentionally reinforce biases present in data, leading to unfair decisions. The automation of jobs also threatens employment in certain industries, necessitating policies for workforce reskilling. Ethical AI development requires transparency, accountability, and human oversight to ensure fairness and safety. AI continues to shape the future, influencing multiple domains. While it brings immense benefits, responsible development and regulation are necessary to harness its potential while mitigating risks.
    """
word2vec_model = api.load('word2vec-google-news-300')
summarizer = BeamSearchSummarizer(text, word2vec_model)
summary = summarizer.beam_search_summary()
print(summary)


In healthcare, AI-powered diagnostic tools have improved the accuracy and speed of disease detection. AI continues to shape the future, influencing multiple domains. It is a rapidly evolving field that integrates machine learning, deep learning, and natural language processing to mimic human intelligence. In the business sector, AI has streamlined operations through automation and predictive analytics. Ethical AI development requires transparency, accountability, and human oversight to ensure fairness and safety.


3. GLOVE

In [15]:
import nltk
import numpy as np
import gensim.downloader as api
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from heapq import nlargest

class BeamSearchSummarizer:
    def __init__(self, text, beam_width=3, summary_length=5):
        nltk.download('punkt')
        self.sentences = nltk.sent_tokenize(text)
        self.beam_width = beam_width
        self.summary_length = summary_length
        self.glove_model = api.load('glove-wiki-gigaword-300')  # Load GloVe embeddings inside the class

    def get_sentence_vector(self, sentence):
        """Convert a sentence into a vector using GloVe embeddings"""
        words = [w for w in nltk.word_tokenize(sentence.lower()) if w.isalpha()]
        vectors = [self.glove_model[word] for word in words if word in self.glove_model]
        return np.mean(vectors, axis=0) if vectors else np.zeros(self.glove_model.vector_size)

    def cluster_sentences(self, num_clusters):
        """Cluster sentences and extract the most important ones"""
        sentence_vectors = np.array([self.get_sentence_vector(sent) for sent in self.sentences])
        kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10).fit(sentence_vectors)
        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_

        key_sentences = []
        for cluster in range(num_clusters):
            indices = np.where(labels == cluster)[0]
            cluster_vectors = sentence_vectors[indices]
            centroid_vector = centroids[cluster].reshape(1, -1)
            similarities = cosine_similarity(cluster_vectors, centroid_vector).flatten()
            top_sentence_index = indices[np.argmax(similarities)]
            key_sentences.append(self.sentences[top_sentence_index])

        return key_sentences

    def beam_search_summary(self):
        """Generate a summary using Beam Search"""
        num_clusters = min(len(self.sentences), self.summary_length)
        key_sentences = self.cluster_sentences(num_clusters)
        beams = [([], 0)]  # Initialize with empty list and score

        for sentence in key_sentences:
            new_beams = []
            for beam_list, beam_score in beams:
                new_list = beam_list + [sentence]
                score = self.evaluate_summary(new_list)
                new_beams.append((new_list, score))
            beams = nlargest(self.beam_width, new_beams, key=lambda x: x[1])

        best_summary = beams[0][0] if beams else []
        return " ".join(best_summary)

    def evaluate_summary(self, summary):
        """Evaluate summaries based on diversity and length constraints"""
        if not summary:
            return 0
        length_penalty = abs(len(summary) - self.summary_length)
        redundancy_penalty = sum(summary.count(s) - 1 for s in set(summary))
        unique_sentences = len(set(summary))
        return unique_sentences - length_penalty - redundancy_penalty

# Example Usage:
text = """
    Artificial Intelligence (AI) has revolutionized various aspects of human life, from automation to decision-making. It is a rapidly evolving field that integrates machine learning, deep learning, and natural language processing to mimic human intelligence. AI has had a significant impact on four key areas: healthcare, business, education, and ethics. In healthcare, AI-powered diagnostic tools have improved the accuracy and speed of disease detection. Machine learning models analyze medical images, detect patterns, and assist doctors in making precise diagnoses. Additionally, AI-driven robotic surgeries and personalized treatment plans are enhancing patient care and medical outcomes. In the business sector, AI has streamlined operations through automation and predictive analytics. Companies use AI-powered chatbots for customer service, fraud detection in banking, and demand forecasting for supply chain management. AI-driven insights help businesses make data-driven decisions, increasing efficiency and profitability. Education has also been transformed by AI, making learning more accessible and personalized. AI-based tutoring systems adapt to individual learning paces, while automated grading reduces the burden on teachers. Virtual reality (VR) and AI-powered simulations provide immersive learning experiences, bridging the gap between theory and practice. Despite its benefits, AI raises ethical concerns regarding privacy, bias, and job displacement. Algorithms can unintentionally reinforce biases present in data, leading to unfair decisions. The automation of jobs also threatens employment in certain industries, necessitating policies for workforce reskilling. Ethical AI development requires transparency, accountability, and human oversight to ensure fairness and safety. AI continues to shape the future, influencing multiple domains. While it brings immense benefits, responsible development and regulation are necessary to harness its potential while mitigating risks.
    """

summarizer = BeamSearchSummarizer(text)
summary = summarizer.beam_search_summary()
print(summary)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Machine learning models analyze medical images, detect patterns, and assist doctors in making precise diagnoses. It is a rapidly evolving field that integrates machine learning, deep learning, and natural language processing to mimic human intelligence. AI has had a significant impact on four key areas: healthcare, business, education, and ethics. While it brings immense benefits, responsible development and regulation are necessary to harness its potential while mitigating risks. Ethical AI development requires transparency, accountability, and human oversight to ensure fairness and safety.


In [16]:
import nltk
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from heapq import nlargest

class BeamSearchSummarizer:
    def __init__(self, text, beam_width=3, summary_length=5):
        nltk.download('punkt')
        self.sentences = nltk.sent_tokenize(text)
        self.beam_width = beam_width
        self.summary_length = summary_length
        self.sbert_model = SentenceTransformer('all-MiniLM-L6-v2')  # Load SBERT model

    def get_sentence_vector(self, sentence):
        """Convert a sentence into a vector using SBERT embeddings"""
        return self.sbert_model.encode(sentence, convert_to_numpy=True)

    def cluster_sentences(self, num_clusters):
        """Cluster sentences and extract the most important ones"""
        sentence_vectors = np.array([self.get_sentence_vector(sent) for sent in self.sentences])
        kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10).fit(sentence_vectors)
        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_

        key_sentences = []
        for cluster in range(num_clusters):
            indices = np.where(labels == cluster)[0]
            cluster_vectors = sentence_vectors[indices]
            centroid_vector = centroids[cluster].reshape(1, -1)
            similarities = cosine_similarity(cluster_vectors, centroid_vector).flatten()
            top_sentence_index = indices[np.argmax(similarities)]
            key_sentences.append(self.sentences[top_sentence_index])

        return key_sentences

    def beam_search_summary(self):
        """Generate a summary using Beam Search"""
        num_clusters = min(len(self.sentences), self.summary_length)
        key_sentences = self.cluster_sentences(num_clusters)
        beams = [([], 0)]  # Initialize with empty list and score

        for sentence in key_sentences:
            new_beams = []
            for beam_list, beam_score in beams:
                new_list = beam_list + [sentence]
                score = self.evaluate_summary(new_list)
                new_beams.append((new_list, score))
            beams = nlargest(self.beam_width, new_beams, key=lambda x: x[1])

        best_summary = beams[0][0] if beams else []
        return " ".join(best_summary)

    def evaluate_summary(self, summary):
        """Evaluate summaries based on diversity and length constraints"""
        if not summary:
            return 0
        length_penalty = abs(len(summary) - self.summary_length)
        redundancy_penalty = sum(summary.count(s) - 1 for s in set(summary))
        unique_sentences = len(set(summary))
        return unique_sentences - length_penalty - redundancy_penalty

# Example Usage:
text = """
    Artificial Intelligence (AI) has revolutionized various aspects of human life, from automation to decision-making. It is a rapidly evolving field that integrates machine learning, deep learning, and natural language processing to mimic human intelligence. AI has had a significant impact on four key areas: healthcare, business, education, and ethics. In healthcare, AI-powered diagnostic tools have improved the accuracy and speed of disease detection. Machine learning models analyze medical images, detect patterns, and assist doctors in making precise diagnoses. Additionally, AI-driven robotic surgeries and personalized treatment plans are enhancing patient care and medical outcomes. In the business sector, AI has streamlined operations through automation and predictive analytics. Companies use AI-powered chatbots for customer service, fraud detection in banking, and demand forecasting for supply chain management. AI-driven insights help businesses make data-driven decisions, increasing efficiency and profitability. Education has also been transformed by AI, making learning more accessible and personalized. AI-based tutoring systems adapt to individual learning paces, while automated grading reduces the burden on teachers. Virtual reality (VR) and AI-powered simulations provide immersive learning experiences, bridging the gap between theory and practice. Despite its benefits, AI raises ethical concerns regarding privacy, bias, and job displacement. Algorithms can unintentionally reinforce biases present in data, leading to unfair decisions. The automation of jobs also threatens employment in certain industries, necessitating policies for workforce reskilling. Ethical AI development requires transparency, accountability, and human oversight to ensure fairness and safety. AI continues to shape the future, influencing multiple domains. While it brings immense benefits, responsible development and regulation are necessary to harness its potential while mitigating risks.
    """

summarizer = BeamSearchSummarizer(text)
summary = summarizer.beam_search_summary()
print(summary)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Despite its benefits, AI raises ethical concerns regarding privacy, bias, and job displacement. Education has also been transformed by AI, making learning more accessible and personalized. In the business sector, AI has streamlined operations through automation and predictive analytics. 
    Artificial Intelligence (AI) has revolutionized various aspects of human life, from automation to decision-making. In healthcare, AI-powered diagnostic tools have improved the accuracy and speed of disease detection.
